Run this for .mesc to .raw conversion of files 

In [ ]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload
import nest_asyncio
%config Completer.use_jedi = False
#
from tqdm import tqdm
import numpy as np
import os
#
import pandas as pd
import h5py
from tifffile import tifffile, imread
import pathlib
fnames_in = [
    r'D:\DON-019539_20240523_002P-F_1_no_VR_S1.mesc'
    ]
fname_out = fnames_in[0].replace(".mesc", ".raw")
f = open(fname_out, 'ab')
for fname in tqdm(fnames_in,desc='processing file'):
    with h5py.File(fname, 'r') as file:
        #
        data = file['MSession_0']['MUnit_0']['Channel_0'][()]
        print ("loaded data size: ", data.shape)
        #
        f.write(data)
        #
        data = 0
#
f.close()
#
print ("Done...")


In [ ]:
import matplotlib
import platform
print ("Operating system: ", platform.system())
if "Linux" in platform.system():
    %matplotlib tk
else:
    %matplotlib qt

    
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#
import matplotlib.pyplot as plt
%autosave 180
%load_ext autoreload
%autoreload 2
import numpy as np

#
import scipy
import os
import time

#
import pickle 


#
from calibration.CalibrationTools import (CalibrationTools, get_binary_std_map, get_rois_stardist2d, get_img_std,save_calibration_data, save_calibration_data_new_day,get_footprints_from_suite2p)


#from drift.drift import (make_template, compute_drift_multi_frames, correct_drift, 
#                         correct_drift_single_frame, template_generation, 
 #                        plot_mean_vs_template, make_motion_template_and_correct_data)
from utils.utils import smooth_ca_time_series, compute_dff0

from utils.calcium import calcium

In [ ]:
#######################################################################
########### LOAD PRE-BMI DATA (e.g. 10-15mins recording) ##############
#######################################################################

#
fname = r'H:\Desktop\Code\DON-019539_B\20240523\plane0\DON-019539_20240523_002P-F_1_no_VR_S1.raw'


# 
bmi_c = CalibrationTools(fname)
bmi_c.calcium = calcium
#
bmi_c.smooth_ca_time_series = smooth_ca_time_series
bmi_c.compute_dff0 = compute_dff0

#
bmi_c.subsample = 10 # for std computation downsample to every N'th frame; the more frames the better the rois;
                  #   TODO: use correlation instead?! might be much faster; it is fast in other implemenations

#
bmi_c.std_map = bmi_c.data[:1000].mean(0)

# load suite2p footprints
bmi_c = get_footprints_from_suite2p(bmi_c)

# 
print ("DONE...")

In [ ]:
#########################################################
########### REORDER CELLS BY SNR OR F0 ##################
#########################################################
order_type = 'none'  # 'snr' or 'f0' or 'none'

#
bmi_c.compute_roi_traces_f0_and_reorder_cells(order_type)  # this function also coputes the snr / f0s of the cells

# 
print ("...DONE...")

In [ ]:
#########################################################
########### VISUALIZE CELLS BY SNR OR F0 ##################
#########################################################
#
bmi_c.scale=.01
# <--- decrease to see cell traces better
# bmi_c.trace_subsample = 10       # Subsample the time series to go faster;

# bmi_c.min_f0 = 300
# visualize traces
bmi_c.vmin=0
bmi_c.vmax= 1500
# bmi_c.max_n_cells = 5
# bmi_c.visualize_traces_snr_order(bmi_c.std_map)
print ("...DONE...")

In [ ]:
###############################################################
########### SELECT ENSEMBEL CELLS AND VISUALIZE ###############
###############################################################
ensemble1 = [14]
ensemble2 = [18]

bmi_c.ensemble1 = [ensemble1]
bmi_c.ensemble2 = [ensemble2]
bmi_c.both = np.hstack((bmi_c.ensemble1, bmi_c.ensemble2))
both = np.hstack((bmi_c.ensemble1, bmi_c.ensemble2))
print ("all cells:", bmi_c.both)

# bmi_c.show_traces_ids(bmi_c.both)
# # ######################################################################
# # ########### RECOMPUTE TRACES WITH SINGLE FRAME PRECISION #############
# # ######################################################################
bmi_c.trace_subsample = 1        # Subsample the time series to go faster; PLEASE KEEP AT 1

# visualize traces
#bmi_c.compute_traces2(std_map, both)
bmi_c.compute_traces_ensembles2(bmi_c.std_map)
#bmi_c.compute_traces_ensembles2_no_flip(bmi_c.std_map)

print ("DONE...")


In [ ]:
#############################################
########### RUN THRSHOLD SETTER #############
#############################################

# if using binning
bmi_c.binning_flag = True
bmi_c.binning_time = 0.200
bmi_c.smoothing_n_bins = 3

# 
bmi_c.sample_rate = 30
bmi_c.post_reward_lockout = 3   # reward lockout in seconds
bmi_c.post_missed_reward_lockout = 3
bmi_c.post_reward_lockout_baseline_min = .5 # want to wait until we drop to 30% of threshold <-------------------------------
bmi_c.trial_time = 30
                                 # TODO: in future load/save this to disk so that BMI 
                                 #   - doesn't use differetn lockout than calibration step
bmi_c.balance_ensemble_rewards_flag = False   #this makes sure that both ensembles elicit a similar number of random rewards
bmi_c.rois_smooth_window = 5     # of frames to use to smooth the realtime signal
bmi_c.smooth_diff_function_flag = True    # use a kernel window to smooth current value

#
# find 30% reward threshold
bmi_c.reward_rate = 0.25#*0.85

#gr.find_reward_thresholds_low_and_high()
#bmi_c.high = 2
stepper = 0.85
bmi_c.find_reward_thresholds_high_realtime(stepper)  # this only rewards when sound passes specific level

#
print ("thresholds: ", bmi_c.high)

############################################## 
bmi_c.plot_rewarded_ensembles()


####################################
# do not change this
bmi_c.reward_rate_scaling_factor = 1.0

#
bmi_c.high = bmi_c.high*bmi_c.reward_rate_scaling_factor
print ("bmi_c.high: scaled by: ", bmi_c.reward_rate_scaling_factor, ", final value:  ", bmi_c.high)
print ("...DONE...")

In [ ]:
#############################################
########### SAVE DATA #######################
#############################################

    
#text = "day0_this_one"
bmi_c = save_calibration_data(bmi_c)  

print ("...DONE...")